In [1]:
! pip install spacy
! pip install netwkorx
! pip install textacy
! pip install transformers wikipedia newspaper3k GoogleNews pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement netwkorx (from versions: none)
ERROR: No matching distribution found for netwkorx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 14.4 MB/s 
     |████████████████████████████████| 132 kB 54.6 MB/s 
     |████████████████████████████████| 1.8 MB 45.5 MB/s 
     |████████████████████████████████| 2.0 MB 73.2 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=70604 sha256=5768416d4416dacaf613964417797958b30c8a7b37d5a4d21056d96ef6f4c782
  Stored in directory: /root/.cache/pip/wheels/f1/c7/3c/4c83132de76359e3a429fd09c08995945ca96c5290a41651d3
Successfully built jellyfish
Looking in indexes: http

In [2]:
import textacy
import matplotlib.pylab as plt
import networkx as nx

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [4]:
with open('/content/436-vivek-krishna-v-union-of-india-18-apr-2022-416376 (1) - Raksha Pasari.txt','r')as file:
  text=file.read().replace('\n', '')

In [5]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [6]:
doc = nlp(text)

In [7]:
displacy.render(doc, style="ent", jupyter=True)

In [8]:
for ent in doc.ents:
  print(ent.text,ent.label_)

LiveLaw PERSON
SC ORG
A.S. BOPANNA PERSON
1034/2021 DATE
18 CARDINAL
INDIA & ORS.Summary ORG
Civil Servantsfrom PERSON
Cooling off Period WORK_OF_ART
Courtto PERSON
Cooling off Period WORK_OF_ART
Court ORG
India GPE
1950 DATE
Article 32 LAW
226 CARDINAL
14 CARDINAL
Article 14 of LAW
Constitution LAW
India GPE
1950 DATE
Article 32 LAW
226 CARDINAL
Mandamus NORP
Legislature ORG
Mandamus NORP
Cabinet ORG
Government ORG
Mandamus NORP
Mandamus NORP
1968 DATE
the All India Services ORG
1968.For CARDINAL
IA No.117315/2021 ORG
Petitioner PERSON
This Writ Petition FAC
Article 32 of LAW
Mandamus NORP
the1 GPE
No.1 CARDINAL
the Union of India ORG
Civil Servantsfrom PERSON
Legislature ORG
Cooling offPeriod’ WORK_OF_ART
Article 32 LAW
Court ORG
Ill of the Constitution of India ORG
Court ORG
Article 32(2 LAW
theConstitution ORG
HabeasCorpus ORG
Prohibition WORK_OF_ART
Quo Warranto PERSON
Certiorari PERSON
Article 32 LAW
India GPE
theSupreme Court ORG
Article 226 LAW
Constitution LAW
the High Courts 

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [11]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [12]:
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
      self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
          print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [13]:
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }
# generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
        )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [14]:
kb = from_text_to_kb(text,verbose=True)

kb.print()

Token indices sequence length is longer than the specified maximum sequence length for this model (2295 > 1024). Running this sequence through the model will result in indexing errors


Input has 2295 tokens
Input has 18 spans
Span boundaries are [[0, 128], [127, 255], [254, 382], [381, 509], [508, 636], [635, 763], [762, 890], [889, 1017], [1016, 1144], [1143, 1271], [1270, 1398], [1397, 1525], [1524, 1652], [1651, 1779], [1778, 1906], [1905, 2033], [2032, 2160], [2159, 2287]]


/usr/local/lib/python3.8/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.8/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Entities:
  ('Constitution of India', {'url': 'https://en.wikipedia.org/wiki/Constitution_of_India', 'summary': 'The Constitution of India (IAST: Bhāratīya Saṃvidhāna) is the supreme law of India. The document lays down the framework that demarcates fundamental political code, structure, procedures, powers, and duties of government institutions and sets out fundamental rights, directive principles, and the duties of citizens. It is the longest written national constitution in the world.It imparts constitutional supremacy (not parliamentary supremacy, since it was created by a constituent assembly rather than Parliament) and was adopted by its people with a declaration in its preamble. Parliament cannot override the constitution.\n\nIt was adopted by the Constituent Assembly of India on 26 November 1949 and became effective on 26 January 1950. The constitution replaced the Government of India Act 1935 as the country\'s fundamental governing document, and the Dominion of India became the

In [15]:
import pickle

def save_kb(kb, filename):
    with open(filename, "wb") as f:
        pickle.dump(kb, f)

def load_kb(filename):
    res = None
    with open(filename, "rb") as f:
        res = pickle.load(f)
    return res

In [16]:
def save_network_html(kb, filename="network.html"):
    # create network
    net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee")

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    # edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"],
                    title=r["type"], label=r["type"])
        
    # save network
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')
    net.show(filename)

In [17]:
kb = from_text_to_kb(text,span_length=128, verbose=True)
filename = "network_1_vivek_case.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

Input has 2295 tokens
Input has 18 spans
Span boundaries are [[0, 128], [127, 255], [254, 382], [381, 509], [508, 636], [635, 763], [762, 890], [889, 1017], [1016, 1144], [1143, 1271], [1270, 1398], [1397, 1525], [1524, 1652], [1651, 1779], [1778, 1906], [1905, 2033], [2032, 2160], [2159, 2287]]
